## Example using partition_networkx

In [1]:
## pip install partition-networkx
## don't forget to 'pip install python-louvain' to get 'community' 

import networkx as nx
import partition_networkx
import numpy as np
import community


In [2]:
# Graph generation with 10 communities of size 100
commSize = 100
numComm = 10
G = nx.generators.planted_partition_graph(l=numComm, k=commSize, p_in=0.1, p_out=0.025)
## store groud truth communities as 'iterables of sets of vertices'
true_comm = [set(list(range(commSize*i, commSize*(i+1)))) for i in range(numComm)]

In [3]:
## run Louvain and ECG:
ml = community.best_partition(G)
ec = community.ecg(G, ens_size=32)

In [4]:
## modularity (w.r.t. original weights for ECG)
print(community.modularity(ml,G),community.modularity(ec.partition,G))

0.19803364963412737 0.2064593397864968


In [5]:
# for 'gam' partition are either iterables of sets of vertices or 'dict'
print("Adjusted Graph-Aware Rand Index for Louvain:",G.gam(true_comm, ml))
print("Adjusted Graph-Aware Rand Index for ecg:",G.gam(true_comm, ec.partition))

print("\nJaccard Graph-Aware for Louvain:",G.gam(true_comm, ml, method="jaccard",adjusted=False))
print("Jaccard Graph-Aware for ecg:",G.gam(true_comm, ec.partition, method="jaccard",adjusted=False))


Adjusted Graph-Aware Rand Index for Louvain: 0.614593615211213
Adjusted Graph-Aware Rand Index for ecg: 0.7681748103383004

Jaccard Graph-Aware for Louvain: 0.5820572312451663
Jaccard Graph-Aware for ecg: 0.7248596222562532


In [6]:
## adjusted RAND index requires iterables over the vertices:
from sklearn.metrics import adjusted_rand_score as ARI
tc = {val:idx for idx,part in enumerate(true_comm) for val in part}

## compute ARI
print("Adjusted non-Graph-Aware Rand Index for Louvain:",ARI(list(tc.values()), list(ml.values())))
print("Adjusted non-Graph-Aware Rand Index for ecg:",ARI(list(tc.values()), list(ec.partition.values())))


Adjusted non-Graph-Aware Rand Index for Louvain: 0.5064603802365473
Adjusted non-Graph-Aware Rand Index for ecg: 0.6831462891054353


In [7]:
## Louvain now also in networkx
lc = nx.community.louvain_communities(G, resolution=1)
lcd = {val:idx for idx,part in enumerate(lc) for val in part}
print('modularity:',community.modularity(lcd,G))
print('ARI:',ARI(list(tc.values()), list(lcd.values())))

modularity: 0.1875501616314812
ARI: 0.610743534328777


## Example using partition_igraph


In [8]:
## pip install partition-igraph
import igraph as ig
import partition_igraph


In [9]:
## previous graph 'G' in igraph format:
g = ig.Graph(directed=False)
g.add_vertices(G.nodes())
g.add_edges(G.edges())

In [10]:
## run Louvain and ECG (with Leiden):
ml = g.community_multilevel()
ec = g.community_ecg(ens_size=32, final='leiden')


In [11]:
## for 'gam' partition are either 'igraph.clustering.VertexClustering' or 'dict'
print('Adjusted Graph-Aware Rand Index for Louvain:',g.gam(ml,tc))
print('Adjusted Graph-Aware Rand Index for ECG:',g.gam(ec,tc))
print('\nJaccard Graph-Aware for Louvain:',g.gam(ml,tc,method="jaccard",adjusted=False))
print('Jaccard Graph-Aware for ECG:',g.gam(ec,tc,method="jaccard",adjusted=False))


Adjusted Graph-Aware Rand Index for Louvain: 0.33142805797101044
Adjusted Graph-Aware Rand Index for ECG: 0.7719212248751264

Jaccard Graph-Aware for Louvain: 0.3704352513138391
Jaccard Graph-Aware for ECG: 0.7290498045215026


In [12]:
## modularity (w.r.t. original weights for ECG)
print(ml.modularity, ec.modularity)

0.17696105402576387 0.20763794138613786
